In [11]:
import os
import time
import re
import chromedriver_autoinstaller
from time import sleep
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm

In [12]:
# 크롬 버젼 확인 후 새로운 드라이버 설치해서 실행
chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]  #크롬드라이버 버전 확인

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito") # 시크릿 모드
chrome_options.add_argument("--headless") # 리눅스 GUI 없는 디스플레이 모드
chrome_options.add_argument("--no-sandbox") # 리소스에 대한 엑서스 방지
chrome_options.add_argument("--disable-setuid-sandbox") # 크롬 충돌 방지
chrome_options.add_argument("--disable-dev-shm-usage") # 메모리 부족 에러 방지
chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])

try: # 크롬 드라이버
    driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver', chrome_options=chrome_options)   
except:
    chromedriver_autoinstaller.install(True)
    driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver', chrome_options=chrome_options)

# WebDruverException Error 방지 기존의 드라이버 버젼으로 지정
# driver = webdriver.Chrome(executable_path='/Users/cmblir/Python/Musinsa-Analysis/100/chromedriver')
driver.implicitly_wait(10) # 10초정도 멈추기

/var/folders/5v/6byb3xz548v1xd0b83z24ys40000gn/T/ipykernel_26296/3550613424.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver', chrome_options=chrome_options)
/var/folders/5v/6byb3xz548v1xd0b83z24ys40000gn/T/ipykernel_26296/3550613424.py:13: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver', chrome_options=chrome_options)


In [13]:
gu_list = ['마포구','서대문구','은평구','종로구','중구','용산구','성동구','광진구',
           '동대문구','성북구','강북구','도봉구','노원구','중랑구','강동구','송파구',
           '강남구','서초구','관악구','동작구','영등포구','금천구','구로구','양천구','강서구']

In [18]:
for index, gu_name in tqdm(enumerate(gu_list)):
    fileName = 'test.csv' # index.__str__() + '_' + gu_name + '.'+'csv'
    file = open(fileName, 'w', encoding='utf-8')
    file.write("카페명" + "|" + "주소" + "|" + "영업시간" + "|" + "전화번호" + "|" + "대표사진주소" + "\n")
    file.close()                                    # 처음에 csv파일에 칼럼명 만들어주기

    driver.get('https://map.kakao.com/')  # 주소 가져오기
    search_area = driver.find_element(By.XPATH, '//*[@id="search.keyword.query"]') # 검색 창
    search_area.send_keys(gu_name + ' 카페')  # 검색어 입력
    driver.find_element(By.XPATH, '//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # Enter로 검색
    driver.implicitly_wait(3) # 기다려 주자
    more_page = driver.find_element(By.ID, "info.search.place.more")
    # more_page.click()
    more_page.send_keys(Keys.ENTER) # 더보기 누르고
    # 첫 번째 검색 페이지 끝
    # driver.implicitly_wait(5) # 기다려 주자
    time.sleep(1)

    # next 사용 가능?
    next_btn = driver.find_element(By.ID, "info.search.page.next")
    has_next = "disabled" not in next_btn.get_attribute("class").split(" ")
    Page = 1
    while has_next: # 다음 페이지가 있으면 loop
    # for i in range(2, 6): # 2, 3, 4, 5
        file = open(fileName, 'a', encoding='utf-8')
        time.sleep(1)
        # place_lists = driver.find_element(By.CSS_SELECTOR, ('#info\.search\.place\.list > li:nth-child(1)')
        # 페이지 루프
        #info\.search\.page\.no1 ~ .no5
        page_links = driver.find_elements(By.CSS_SELECTOR, "#info\.search\.page a")
        pages = [link for link in page_links if "HIDDEN" not in link.get_attribute("class").split(" ")]
        # print(len(pages), "개의 페이지 있음")
        # pages를 하나씩 클릭하면서
        for i in range(1, 6):
            xPath = '//*[@id="info.search.page.no' + str(i) + '"]'
            try:
                page = driver.find_element(By.XPATH, xPath)
                page.send_keys(Keys.ENTER)
            except ElementNotInteractableException:
                print('End of Page')
                break;
            sleep(3)
            place_lists = driver.find_elements(By.CSS_SELECTOR, '#info\.search\.place\.list > li')
            for p in place_lists: # WebElement
                # print(p.get_attribute('innerHTML'))
                # print("type of p:", type(p))
                store_html = p.get_attribute('innerHTML')
                store_info = BeautifulSoup(store_html, "html.parser")
                # BS -> 분석
                #
                place_name = store_info.select('.head_item > .tit_name > .link_name')
                # place_address = store_info.select('.info_item > .addr > p')
                # place_hour = store_info.select('.info_item > .openhour > p > a')
                # place_tel = store_info.select('.info_item > .contact > span')
                 # print("length:", len(place_name))
                if len(place_name) == 0:
                    continue # 광고
                place_name = store_info.select('.head_item > .tit_name > .link_name')[0].text
                place_address = store_info.select('.info_item > .addr > p')[0].text
                place_hour = store_info.select('.info_item > .openhour > p > a')[0].text
                place_tel = store_info.select('.info_item > .contact > span')[0].text


                # 사진url 수집
                detail = p.find_element(By.CSS_SELECTOR, 'div.info_item > div.contact > a.moreview')
                detail.send_keys(Keys.ENTER)

                driver.switch_to.window(driver.window_handles[-1])

                place_photo = ""
                try:
                    photo = driver.find_elements(By.CSS_SELECTOR, 'span.bg_present')
                    photo_url = photo.get_attribute('style')
                    m = re.search('"(.+?)"', photo_url)
                    if m:
                        place_photo = m.group(1)
                    else:
                        place_photo = ""
                except:
                    place_photo = ""
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
                print(place_name, place_photo)

                file.write(place_name + "|" + place_address + "|" + place_hour + "|" + place_tel + "|" + place_photo + "\n")
            print(i, ' of', ' [ ' , Page, ' ] ')
        next_btn = driver.find_element(By.ID, "info.search.page.next")
        has_next = "disabled" not in next_btn.get_attribute("class").split(" ")
        if not has_next:
            print('Arrow is Disabled')
            driver.close()
            file.close()
            break # 다음 페이지 없으니까 종료
        else: # 다음 페이지 있으면
            Page += 1
            next_btn.send_keys(Keys.ENTER)
    print("End of Crawl")

0it [00:00, ?it/s]

어반플랜트 합정 
피오니 홍대점 
프릳츠 도화점 
943킹스크로스 
카페공명 
오츠커피 마포점 
카페 레이어드 연남점 
비파티세리 공덕점 
앤트러사이트 합정점 
포비 합정점 
앤트러사이트 서교점 
테일러커피 서교점 
당인리책발전소 
무신사테라스 
톤앤매너 
1  of  [  1  ] 
커피리브레 연남점 
1984 
빈브라더스 합정점 
땡스오트 연남점 
오랑지 
콜린 
디벙크 
땡스네이쳐 
얼스어스 
샌드커피 논탄토 
키쉬미뇽 
커피가게동경 
브런치가 
아이엠베이글 공덕점 
테일러커피 연남1호점 
2  of  [  1  ] 
작당모의 
딥블루레이크 
이미커피 
버터밀크 
시루케이크 
콩카페 연남점 
빌리프커피로스터스 
W카페 
무대륙 
비트포비아 홍대던전점 
82년생김민경X카페르세 
허밍벨라 
모아새 
딩가케이크 
제비다방 
3  of  [  1  ] 
더페이머스램 
카페스콘 
연남방앗간 
씨스루 홍대연남점 
포멜로빈 공덕점 
밤부베이커리&브루잉 
히어로보드게임카페 홍대1호점 
카페 클로이인패리스 
그레이랩 
카페 지금여기 
카페장쌤 
앤디스커피 
그로토 
서울이스케이프룸 홍대2호점 
공상온도 
4  of  [  1  ] 
노티드 연남 
라헬의부엌 홍대점 
브루라티오 홍대점 
비로소커피 
랑데자뷰 상수점 
광합성카페 
ESC방탈출카페 홍대점 
민트하임 
도쿄빙수 본점 
커피폴리 
스탠스커피 
멜로우 
미스터리룸이스케이프 홍대2호점 
네시사분 
퍼셉션 (Perception) 
5  of  [  1  ] 
8810리스트레토바 
산리오러버스클럽 
달콤한거짓말 
몽마르뜨언덕위 은하수다방 
이리카페 
펠른 
카페여유 
히어로보드게임카페 홍대2호점 
카페105 
저스티나 
르쁘띠푸 홍대본점 
딥커피 
모파상 
커피냅로스터스 연남동 
카페 사운드웨이브 
1  of  [  2  ] 
티크닉 
꼬르소산도 
어니스트팬케이크 
애프터눈커피 
딩굴딩굴알타미라 
카페로제 
재미난조각가 홍대점 
미어캣프랜즈 
17도씨 
금옥당 서교점 
미완성식탁 
비포블루밍 
다다랩 
리플로우 연남점 

1it [59:26, 3566.62s/it]

End of Page
Arrow is Disabled
End of Crawl


InvalidSessionIdException: Message: invalid session id
Stacktrace:
0   chromedriver                        0x0000000104febf38 chromedriver + 4910904
1   chromedriver                        0x0000000104f6ba03 chromedriver + 4385283
2   chromedriver                        0x0000000104bb05bd chromedriver + 472509
3   chromedriver                        0x0000000104be498f chromedriver + 686479
4   chromedriver                        0x0000000104c1b016 chromedriver + 909334
5   chromedriver                        0x0000000104c16edd chromedriver + 892637
6   chromedriver                        0x0000000104c161ff chromedriver + 889343
7   chromedriver                        0x0000000104b7bafa chromedriver + 256762
8   chromedriver                        0x0000000104fb91de chromedriver + 4702686
9   chromedriver                        0x0000000104fbdb19 chromedriver + 4721433
10  chromedriver                        0x0000000104fc528e chromedriver + 4752014
11  chromedriver                     